# 地质信息

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# all columns
# name scheme: columns-num_cycles-sl
from config import *
context = Context('validation', gpu_start=6, dep_var=['桩号'], debug=0, mulr=1, load_data=False)
from fastai.callbacks.mem import preload_pytorch

In [ ]:
geo_raw = pd.read_csv('/home/wenxiang/datasets/tbm/geo_csv/geo2.1.csv', index_col=0)

In [ ]:
display_all(geo_raw)

In [ ]:
geo_cols = ['起始桩号', '最后桩号', '实际开挖围岩类别', '洞室部位主要岩性']
geo = geo_raw.loc[:,geo_cols]

In [ ]:
# clean nan
geo = geo.loc[geo.iloc[:,-1].notna()]

In [ ]:
geo.iloc[:,-2].unique(),geo.iloc[:,-1].unique()

In [ ]:
geo.columns

# generate training data with these

In [ ]:
context = Context('amended-3-7', debug=0, dep_var=dep_var2, mulr=7, gpu_start=9)

In [ ]:
zh = context.cyc_cont.iloc[:,-5]

In [ ]:
cat_names = geo.columns.tolist()

In [ ]:
idx_dep_var3 = []
missing_idx = -1 #geo.shape[0] # last item + 1
for z in list(zh):
    is_within = ((z - geo.iloc[:,0]) * (z - geo.iloc[:,1])) < 0
    if is_within.any():
        idx_dep_var3.append(np.argmax(is_within.values))
    else:
        idx_dep_var3.append(missing_idx)

In [ ]:
idx_dep_var3[:10]

In [ ]:
# append a default entry
# geo.iloc[-1,2:].tolist()

In [ ]:
dep_var3 = []
for i in idx_dep_var3:
    dep_var3.append(geo.iloc[i,2:].tolist() if i != -1 else ['oor', 'oor'])

In [ ]:
cycles = pd.concat([context.cyc_cont, pd.DataFrame(dep_var3, columns=['围岩等级', '岩性'])], axis=1)

In [ ]:
# simplify
cat_1 = cycles.loc[:,cat_names[0]]
III_idx = np.logical_or(cat_1 == 'Ⅲa', cat_1 == 'Ⅲb')
cat_1.loc[III_idx] = 'Ⅲ'

IV_idx = np.logical_or(cat_1 == 'Ⅳ好', cat_1 == 'Ⅳ差')
cat_1.loc[IV_idx] = 'Ⅳ'

In [ ]:
cat_names = cycles.columns[-2:]
cont_names = cycles.columns[:-2]

In [ ]:
cf = Categorify(cat_names, cont_names)
cf.apply_train(cycles)

In [ ]:
cycles.iloc[:,-2].cat.codes.unique(), cycles.iloc[:,-1].cat.codes.unique()